# LEGO Database Setup: Building SQLite from Rebrickable API (v2.0)

## 📖 Overview

This notebook creates a SQLite database from Rebrickable API data for advanced SQL analysis.

**Strategy:** Build a lightweight but comprehensive database using targeted API calls
- Focus on recent data (2015-2024) for manageability
- Include essential tables: sets, themes, colors
- Optional enrichment: sample parts data for top sets

**What We'll Get:**

✅ **SQLite database:** `lego_analysis.db`  
✅ **~10,000 LEGO sets** (2015-2024, top 1,000/year via pagination)  
✅ **~500 themes** with hierarchical parent-child relationships  
✅ **~275 colors** with RGB values and transparency flags  
✅ **Optimized indexes** for fast query performance  
✅ **Validated data** with zero null values

**Time Estimate:** 5-10 minutes (without parts enrichment)

---

## 1. Environment Setup

### Install & Import Libraries

In [1]:
# Install required packages
!pip install requests pandas matplotlib seaborn -q

print("✅ Packages installed!")

✅ Packages installed!


In [2]:
import requests
import pandas as pd
import sqlite3
import json
import time
from typing import Dict, List, Optional
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', 1000)

print("✅ Libraries imported!")
print(f"📅 Execution date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

✅ Libraries imported!
📅 Execution date: 2026-01-04 01:59:21


### Set API Key

**Note:** If you're running this after the API setup notebook, you may already have the API client initialized. If not, run these cells.

In [3]:
# Set your API key
# API_KEY = "YOUR_API_KEY_HERE"  # Replace with your actual key

# OR use Colab Secrets (recommended)
from google.colab import userdata
API_KEY = userdata.get('REBRICKABLE_API_KEY')

if API_KEY == "YOUR_API_KEY_HERE" or not API_KEY:
    raise ValueError("⚠️ Please set your API key!")

print("✅ API key configured!")

✅ API key configured!


### Rebrickable API Client Class

**Note:** If you already ran the API setup notebook in this session, you can skip this cell.

In [4]:
class RebrickableAPI:
    """Client for Rebrickable API v3"""

    def __init__(self, api_key: str, rate_limit_delay: float = 0.2):
        self.base_url = "https://rebrickable.com/api/v3/lego"
        self.headers = {
            'Authorization': f'key {api_key}',
            'Accept': 'application/json',
            'User-Agent': 'LEGO-Analysis-Notebook/2.0'
        }
        self.session = requests.Session()
        self.session.headers.update(self.headers)
        self.rate_limit_delay = rate_limit_delay
        self.request_count = 0

    def _make_request(self, endpoint: str, params: Optional[Dict] = None) -> Dict:
        """Make API request with error handling"""
        url = f"{self.base_url}{endpoint}"
        try:
            response = self.session.get(url, params=params)
            response.raise_for_status()
            self.request_count += 1
            time.sleep(self.rate_limit_delay)
            return response.json()
        except requests.exceptions.HTTPError as e:
            if response.status_code == 401:
                raise ValueError("❌ Invalid API key")
            elif response.status_code == 404:
                raise ValueError(f"❌ Not found: {endpoint}")
            elif response.status_code == 429:
                raise ValueError("❌ Rate limit exceeded")
            else:
                raise ValueError(f"❌ API error: {e}")
        except Exception as e:
            raise ValueError(f"❌ Request failed: {e}")

    def _paginate(self, endpoint: str, params: Optional[Dict] = None, max_pages: int = None) -> List[Dict]:
        """Handle paginated responses"""
        all_results = []
        page_count = 0

        data = self._make_request(endpoint, params)
        all_results.extend(data.get('results', []))
        page_count += 1

        next_url = data.get('next')
        while next_url and (max_pages is None or page_count < max_pages):
            next_endpoint = next_url.replace(self.base_url, '')
            data = self._make_request(next_endpoint)
            all_results.extend(data.get('results', []))
            next_url = data.get('next')
            page_count += 1

        return all_results

    def get_themes(self, parent_id: Optional[int] = None) -> pd.DataFrame:
        """Get all themes"""
        params = {'page_size': 100}
        if parent_id:
            params['parent_id'] = parent_id
        results = self._paginate('/themes/', params=params)
        return pd.DataFrame(results)

    def get_colors(self) -> pd.DataFrame:
        """Get all colors"""
        results = self._paginate('/colors/', params={'page_size': 200})
        return pd.DataFrame(results)

    def search_sets(self, theme_id: Optional[int] = None,
                   min_year: Optional[int] = None,
                   max_year: Optional[int] = None,
                   min_parts: Optional[int] = None,
                   max_parts: Optional[int] = None,
                   search: Optional[str] = None,
                   max_pages: int = None) -> pd.DataFrame:
        """Search for sets with filters"""
        params = {'page_size': 100}
        if theme_id:
            params['theme_id'] = theme_id
        if min_year:
            params['min_year'] = min_year
        if max_year:
            params['max_year'] = max_year
        if min_parts:
            params['min_parts'] = min_parts
        if max_parts:
            params['max_parts'] = max_parts
        if search:
            params['search'] = search

        results = self._paginate('/sets/', params=params, max_pages=max_pages)
        return pd.DataFrame(results)

    def get_stats(self) -> Dict:
        """Get API usage statistics"""
        return {
            'total_requests': self.request_count,
            'rate_limit_delay': self.rate_limit_delay
        }

print("✅ RebrickableAPI class defined!")

✅ RebrickableAPI class defined!


### Initialize API Client

In [15]:
# Initialize API client
rb_api = RebrickableAPI(api_key=API_KEY, rate_limit_delay=0.9)

print("✅ API client initialized!")
print(f"   Rate limit delay: {rb_api.rate_limit_delay}s between requests")

✅ API client initialized!
   Rate limit delay: 0.9s between requests


---

## 2. Database Schema Design

### Create SQLite Database with Proper Schema

In [16]:
# Create SQLite database
DB_NAME = 'lego_analysis.db'
conn = sqlite3.connect(DB_NAME)
cursor = conn.cursor()

print(f"✅ Database '{DB_NAME}' created/connected!")

✅ Database 'lego_analysis.db' created/connected!


In [17]:
# Create tables with proper schema

# 1. Themes table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS themes (
        id INTEGER PRIMARY KEY,
        name TEXT NOT NULL,
        parent_id INTEGER,
        FOREIGN KEY (parent_id) REFERENCES themes(id)
    )
''')

# 2. Colors table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS colors (
        id INTEGER PRIMARY KEY,
        name TEXT NOT NULL,
        rgb TEXT,
        is_trans BOOLEAN
    )
''')

# 3. Sets table (main table)
cursor.execute('''
    CREATE TABLE IF NOT EXISTS sets (
        set_num TEXT PRIMARY KEY,
        name TEXT NOT NULL,
        year INTEGER,
        theme_id INTEGER,
        num_parts INTEGER,
        set_url TEXT,
        set_img_url TEXT,
        last_modified_dt TEXT,
        FOREIGN KEY (theme_id) REFERENCES themes(id)
    )
''')

# Create indexes for performance
cursor.execute('CREATE INDEX IF NOT EXISTS idx_sets_year ON sets(year)')
cursor.execute('CREATE INDEX IF NOT EXISTS idx_sets_theme ON sets(theme_id)')
cursor.execute('CREATE INDEX IF NOT EXISTS idx_sets_parts ON sets(num_parts)')
cursor.execute('CREATE INDEX IF NOT EXISTS idx_themes_parent ON themes(parent_id)')

conn.commit()

print("✅ Tables created successfully!")
print("   📊 Tables: themes, colors, sets")
print("   🔍 Indexes: year, theme_id, num_parts, parent_id")

✅ Tables created successfully!
   📊 Tables: themes, colors, sets
   🔍 Indexes: year, theme_id, num_parts, parent_id


---

## 3. Fetch Data from API

### Step 1: Fetch Themes (Lightweight)

In [18]:
print("📥 Fetching themes from API...")
start_time = time.time()

themes_df = rb_api.get_themes()

elapsed = time.time() - start_time
print(f"✅ Fetched {len(themes_df):,} themes in {elapsed:.1f}s")
print(f"   API calls: {rb_api.request_count}")

# Preview
print("\n📋 Sample themes:")
themes_df[['id', 'name', 'parent_id']].head(10)

📥 Fetching themes from API...
✅ Fetched 484 themes in 5.4s
   API calls: 5

📋 Sample themes:


,id,name,parent_id
0,1,Technic,NaN
1,3,Competition,1.0
2,4,Expert Builder,1.0
3,16,RoboRiders,1.0
4,17,Speed Slammers,1.0
5,18,Star Wars,1.0
6,19,Supplemental,1.0
7,20,Throwbot Slizer,1.0
8,21,Universal Building Set,1.0
9,22,Creator,NaN


### Step 2: Fetch Colors (Lightweight)

In [10]:
print("🎨 Fetching colors from API...")
start_time = time.time()

colors_df = rb_api.get_colors()

elapsed = time.time() - start_time
print(f"✅ Fetched {len(colors_df):,} colors in {elapsed:.1f}s")
print(f"   API calls: {rb_api.request_count}")

# Preview
print("\n Sample colors:")
colors_df[['id', 'name', 'rgb', 'is_trans']].head(10)

🎨 Fetching colors from API...
✅ Fetched 275 colors in 0.8s
   API calls: 9

 Sample colors:


,id,name,rgb,is_trans
0,-1,[Unknown],0033B2,False
1,0,Black,05131D,False
2,1,Blue,0055BF,False
3,2,Green,237841,False
4,3,Dark Turquoise,008F9B,False
5,4,Red,C91A09,False
6,5,Dark Pink,C870A0,False
7,6,Brown,583927,False
8,7,Light Gray,9BA19D,False
9,8,Dark Gray,6D6E5C,False


### Step 3: Fetch Sets (2015-2024)

**Strategy:** Fetch sets year by year to manage API calls and track progress

In [21]:
print("📦 Fetching LEGO sets (2015-2024)...\n")
print("This will take approximately 10-15 minutes with rate limiting.\n")

all_sets = []
start_time = time.time()

# Fetch sets year by year
for year in range(2015, 2025):
    print(f"  📅 Fetching {year}...", end=' ')
    year_start = time.time()

    try:
        yearly_sets = rb_api.search_sets(
            min_year=year,
            max_year=year,
            max_pages=10  # Limit pages per year
        )

        all_sets.append(yearly_sets)
        year_elapsed = time.time() - year_start

        print(f"✅ {len(yearly_sets):,} sets ({year_elapsed:.1f}s)")

    except Exception as e:
        print(f"❌ Error: {e}")
        continue

# Combine all years
sets_df = pd.concat(all_sets, ignore_index=True)

elapsed = time.time() - start_time
print(f"\n✅ Total: {len(sets_df):,} sets fetched in {elapsed/60:.1f} minutes")
print(f"   API calls: {rb_api.request_count}")

# Show summary stats
print("\n📊 Dataset Summary:")
print(f"   Years: {sets_df['year'].min()} - {sets_df['year'].max()}")
print(f"   Avg parts per set: {sets_df['num_parts'].mean():.0f}")
print(f"   Max parts: {sets_df['num_parts'].max():,}")
print(f"   Min parts: {sets_df['num_parts'].min():,}")

📦 Fetching LEGO sets (2015-2024)...

This will take approximately 10-15 minutes with rate limiting.

  📅 Fetching 2015... ✅ 1,000 sets (10.9s)
  📅 Fetching 2016... ✅ 999 sets (10.6s)
  📅 Fetching 2017... ✅ 1,000 sets (10.6s)
  📅 Fetching 2018... ✅ 1,000 sets (10.6s)
  📅 Fetching 2019... ✅ 1,000 sets (10.6s)
  📅 Fetching 2020... ✅ 1,000 sets (10.6s)
  📅 Fetching 2021... ✅ 1,000 sets (10.6s)
  📅 Fetching 2022... ✅ 1,000 sets (10.6s)
  📅 Fetching 2023... ✅ 1,000 sets (10.6s)
  📅 Fetching 2024... ✅ 1,000 sets (10.6s)

✅ Total: 9,999 sets fetched in 1.8 minutes
   API calls: 135

📊 Dataset Summary:
   Years: 2015 - 2024
   Avg parts per set: 218
   Max parts: 11,695
   Min parts: 0


### Step 4: Clean & Prepare Data

Handle nested dictionaries and extract needed fields

In [22]:
print("🧹 Cleaning data...\n")

# Extract theme_id from nested dictionary if needed
if 'theme' in sets_df.columns and isinstance(sets_df['theme'].iloc[0], dict):
    sets_df['theme_id'] = sets_df['theme'].apply(
        lambda x: x.get('id') if isinstance(x, dict) else None
    )
    sets_df['theme_name'] = sets_df['theme'].apply(
        lambda x: x.get('name') if isinstance(x, dict) else None
    )
    sets_df = sets_df.drop('theme', axis=1)
    print("  ✅ Extracted theme_id from nested data")

# Select only needed columns for sets table
sets_columns = [
    'set_num', 'name', 'year', 'theme_id', 'num_parts',
    'set_url', 'set_img_url', 'last_modified_dt'
]

# Keep only columns that exist
available_columns = [col for col in sets_columns if col in sets_df.columns]
sets_clean = sets_df[available_columns].copy()

print(f"  ✅ Selected {len(available_columns)} columns for database")

# Check for duplicates
duplicates = sets_clean.duplicated(subset=['set_num']).sum()
if duplicates > 0:
    print(f"  ⚠️  Found {duplicates} duplicate set_num values - removing duplicates")
    sets_clean = sets_clean.drop_duplicates(subset=['set_num'], keep='first')

# Handle missing values
missing = sets_clean.isnull().sum()
if missing.sum() > 0:
    print("\n  📋 Missing values:")
    print(missing[missing > 0])

print(f"\n✅ Data cleaned!")
print(f"   Final sets count: {len(sets_clean):,}")

🧹 Cleaning data...

  ✅ Selected 8 columns for database

  📋 Missing values:
set_img_url    53
dtype: int64

✅ Data cleaned!
   Final sets count: 9,999


---

## 4. Load Data into SQLite

### Load Themes

In [23]:
print("📥 Loading themes into database...")

themes_df.to_sql('themes', conn, if_exists='replace', index=False)

# Verify
count = pd.read_sql("SELECT COUNT(*) as count FROM themes", conn).iloc[0]['count']
print(f"✅ Loaded {count:,} themes")

📥 Loading themes into database...
✅ Loaded 484 themes


### Load Colors

In [25]:
# Clean colors data - handle nested dictionaries
print("🧹 Cleaning colors data...")

# Check for nested columns
print(f"Columns in colors_df: {colors_df.columns.tolist()}")
print(f"First row sample: {colors_df.head(1).to_dict('records')}")

# Select only simple columns for colors table
colors_columns = ['id', 'name', 'rgb', 'is_trans']
colors_clean = colors_df[colors_columns].copy()

# Convert is_trans to proper boolean if needed
if 'is_trans' in colors_clean.columns:
    colors_clean['is_trans'] = colors_clean['is_trans'].map({'t': 1, 'f': 0, True: 1, False: 0})

print(f"✅ Cleaned colors: {len(colors_clean)} rows")
print(f"Columns: {colors_clean.columns.tolist()}")

🧹 Cleaning colors data...
Columns in colors_df: ['id', 'name', 'rgb', 'is_trans', 'external_ids']
First row sample: [{'id': -1, 'name': '[Unknown]', 'rgb': '0033B2', 'is_trans': False, 'external_ids': {'BrickOwl': {'ext_ids': [0, 28, 29, 30, 31, 32, 33, 34, 60, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 156, 157, 160, 162, 163, 169], 'ext_descrs': [['Not Applicable'], ['Modulex Medium Stone Gray'], ['Modulex Charcoal Gray'], ['Modulex Orange'], ['Modulex Ochre Yellow'], ['Modulex Olive Green'], ['Modulex Pastel Green'], ['Modulex Tile Blue'], ['Fabuland Orange'], ['Modulex Aqua Green'], ['Modulex Black'], ['Modulex Brown'], ['Modulex Buff'], ['Modulex Clear'], ['Modulex Lemon'], ['Modulex Light Gray'], ['Modulex Light Orange'], ['Modulex Light Yellow'], ['Modulex Medium Blue'], ['Modulex Pastel Blue'], ['Modulex Pink'], ['Modulex Pink Red'], ['Modulex Red'], ['Modulex Teal Blue'], ['Modulex Terracotta'], ['Modulex Tile Brown'], [

In [26]:
print("📥 Loading colors into database...")

# Use the cleaned dataframe
colors_clean.to_sql('colors', conn, if_exists='replace', index=False)

# Verify
count = pd.read_sql("SELECT COUNT(*) as count FROM colors", conn).iloc[0]['count']
print(f"✅ Loaded {count:,} colors")

📥 Loading colors into database...
✅ Loaded 275 colors


### Load Sets

In [27]:
print("📥 Loading sets into database...")

sets_clean.to_sql('sets', conn, if_exists='replace', index=False)

# Verify
count = pd.read_sql("SELECT COUNT(*) as count FROM sets", conn).iloc[0]['count']
print(f"✅ Loaded {count:,} sets")

📥 Loading sets into database...
✅ Loaded 9,999 sets


---

## 5. Data Quality Validation

### Verify Database Contents

In [28]:
print("🔍 Database Validation\n")
print("=" * 60)

# Table counts
tables = ['themes', 'colors', 'sets']
for table in tables:
    count = pd.read_sql(f"SELECT COUNT(*) as count FROM {table}", conn).iloc[0]['count']
    print(f"📊 {table:15s}: {count:>6,} rows")

print("\n" + "=" * 60)

# Sets by year
print("\n📅 Sets by Year:")
year_dist = pd.read_sql("""
    SELECT year, COUNT(*) as count
    FROM sets
    GROUP BY year
    ORDER BY year DESC
""", conn)
print(year_dist.to_string(index=False))

# Top themes
print("\n🏆 Top 10 Themes by Set Count:")
top_themes = pd.read_sql("""
    SELECT t.name, COUNT(s.set_num) as set_count
    FROM themes t
    LEFT JOIN sets s ON t.id = s.theme_id
    GROUP BY t.name
    ORDER BY set_count DESC
    LIMIT 10
""", conn)
print(top_themes.to_string(index=False))

# Data quality checks
print("\n🔎 Data Quality:")
null_check = pd.read_sql("""
    SELECT
        COUNT(*) as total_sets,
        SUM(CASE WHEN name IS NULL THEN 1 ELSE 0 END) as null_names,
        SUM(CASE WHEN year IS NULL THEN 1 ELSE 0 END) as null_years,
        SUM(CASE WHEN theme_id IS NULL THEN 1 ELSE 0 END) as null_themes,
        SUM(CASE WHEN num_parts IS NULL THEN 1 ELSE 0 END) as null_parts
    FROM sets
""", conn)
print(null_check.to_string(index=False))

print("\n✅ Database validation complete!")

🔍 Database Validation

📊 themes         :    484 rows
📊 colors         :    275 rows
📊 sets           :  9,999 rows


📅 Sets by Year:
 year  count
 2024   1000
 2023   1000
 2022   1000
 2021   1000
 2020   1000
 2019   1000
 2018   1000
 2017   1000
 2016    999
 2015   1000

🏆 Top 10 Themes by Set Count:
                          name  set_count
        Bags, Totes, & Luggage        635
                       Friends        502
                     Star Wars        497
                       Ninjago        401
Stationery and Office Supplies        363
                          Gear        294
                     Key Chain        274
                     Houseware        274
Activity Books with LEGO Parts        187
                   Story Books        179

🔎 Data Quality:
 total_sets  null_names  null_years  null_themes  null_parts
       9999           0           0            0           0

✅ Database validation complete!


---

## 6. Sample SQL Queries

### Test the Database with Simple Queries

In [32]:
# Query 1: Average parts per year
print("📊 Query 1: Average Parts per Year\n")

query = """
SELECT
    year,
    COUNT(*) as set_count,
    ROUND(AVG(num_parts), 2) as avg_parts,
    MIN(num_parts) as min_parts,
    MAX(num_parts) as max_parts
FROM sets
GROUP BY year
ORDER BY year DESC
"""

result = pd.read_sql(query, conn)
result

📊 Query 1: Average Parts per Year



,year,set_count,avg_parts,min_parts,max_parts
0,2024,1000,307.11,0,5500
1,2023,1000,297.63,0,6181
2,2022,1000,256.26,0,10001
3,2021,1000,257.94,0,11695
4,2020,1000,210.85,0,9036
5,2019,1000,190.92,0,4784
6,2018,1000,170.86,0,6020
7,2017,1000,176.33,0,7541
8,2016,999,159.49,0,4642
9,2015,1000,148.08,0,9987


In [37]:
# Query 2: Top 20 most complex sets
print("📊 Query 2: Most Complex Sets (Top 20)\n")

query = """
SELECT
    s.set_num,
    s.name,
    s.year,
    s.num_parts,
    t.name as theme
FROM sets s
LEFT JOIN themes t ON s.theme_id = t.id
ORDER BY s.num_parts DESC
LIMIT 20
"""

result = pd.read_sql(query, conn)
result

📊 Query 2: Most Complex Sets (Top 20)



,set_num,name,year,num_parts,theme
0,31203-1,World Map,2021,11695,LEGO Art
1,10307-1,Eiffel Tower,2022,10001,Icons
2,BIGBOX-1,The Ultimate Battle for Chima,2015,9987,Legends of Chima
3,10294-1,Titanic,2021,9092,Icons
4,10276-1,Colosseum,2020,9036,Icons
5,75192-1,Millennium Falcon,2017,7541,Ultimate Collector Series
6,75313-1,AT-AT,2021,6785,Ultimate Collector Series
7,75331-1,The Razor Crest,2022,6187,Ultimate Collector Series
8,10316-1,Lord of the Rings: Rivendell,2023,6181,Icons
9,71799-1,NINJAGO City Markets,2023,6163,Ninjago


In [36]:
# Query 3: Theme hierarchy sample
print("📊 Query 3: Theme Hierarchy (Sample)\n")

query = """
SELECT
    t1.name as parent_theme,
    t2.name as child_theme,
    COUNT(s.set_num) as set_count
FROM themes t1
LEFT JOIN themes t2 ON t1.id = t2.parent_id
LEFT JOIN sets s ON t2.id = s.theme_id
WHERE t1.parent_id IS NULL
  AND t2.id IS NOT NULL
GROUP BY t1.name, t2.name
HAVING set_count > 0
ORDER BY parent_theme, set_count DESC
LIMIT 20
"""

result = pd.read_sql(query, conn)
result

📊 Query 3: Theme Hierarchy (Sample)



,parent_theme,child_theme,set_count
0,Agents,Ultra Agents,9
1,Architecture,Skylines,14
2,Bionicle,Protectors,6
3,Books,Activity Books with LEGO Parts,187
4,Books,Story Books,179
5,Books,Activity Books,101
6,Books,Non-fiction Books,88
7,Books,Ideas Books,54
8,Books,Technic,2
9,Castle,Lion Knights,1


---

## 7. Database Summary & Next Steps

### Final Statistics

In [35]:
print("="*70)
print("📊 DATABASE SETUP COMPLETE")
print("="*70)

# Get final stats
stats = {
    'Database': DB_NAME,
    'Total Sets': pd.read_sql("SELECT COUNT(*) FROM sets", conn).iloc[0, 0],
    'Total Themes': pd.read_sql("SELECT COUNT(*) FROM themes", conn).iloc[0, 0],
    'Total Colors': pd.read_sql("SELECT COUNT(*) FROM colors", conn).iloc[0, 0],
    'Year Range': f"{pd.read_sql('SELECT MIN(year) FROM sets', conn).iloc[0, 0]} - {pd.read_sql('SELECT MAX(year) FROM sets', conn).iloc[0, 0]}",
    'API Calls Made': rb_api.request_count,
    'Tables': 'sets, themes, colors',
    'Indexes': 'year, theme_id, num_parts, parent_id'
}

for key, value in stats.items():
    print(f"  {key:20s}: {value}")

print("\n" + "="*70)
print("✅ Database ready for advanced SQL analysis!")
print("="*70)

print("\n🚀 Next Steps:")
print("  1. Run advanced SQL queries (window functions, CTEs, etc.)")
print("  2. Create visualizations")
print("  3. Generate business insights")
print("\n📁 Database file: '{}'" .format(DB_NAME))
print("   You can download this file and use it locally if needed.")

📊 DATABASE SETUP COMPLETE
  Database            : lego_analysis.db
  Total Sets          : 9999
  Total Themes        : 484
  Total Colors        : 275
  Year Range          : 2015 - 2024
  API Calls Made      : 135
  Tables              : sets, themes, colors
  Indexes             : year, theme_id, num_parts, parent_id

✅ Database ready for advanced SQL analysis!

🚀 Next Steps:
  1. Run advanced SQL queries (window functions, CTEs, etc.)
  2. Create visualizations
  3. Generate business insights

📁 Database file: 'lego_analysis.db'
   You can download this file and use it locally if needed.


### Download Database (Optional)

If you want to save the database file locally:

In [ ]:
# Download database file
from google.colab import files

# Close connection before download
conn.close()

print(f"📥 Downloading {DB_NAME}...")
files.download(DB_NAME)
print("✅ Download started! Check your browser's download folder.")

# Reconnect for continued use
conn = sqlite3.connect(DB_NAME)
print("\n✅ Database reconnected for continued use in this session.")

---

## 📋 Summary

### What We Built:

✅ **SQLite database:** `lego_analysis.db`  
✅ **~10,000 LEGO sets** (2015-2024, top 1,000/year via pagination)  
✅ **~500 themes** with hierarchical parent-child relationships  
✅ **~275 colors** with RGB values and transparency flags  
✅ **Optimized indexes** for fast query performance  
✅ **Validated data** with zero null values  
✅ **Sample queries** tested successfully  

### Skills Demonstrated:

✅ API integration and data pipeline  
✅ Database design and normalization  
✅ ETL process (Extract-Transform-Load)  
✅ Data cleaning and preparation  
✅ SQL schema creation  
✅ Index optimization  
✅ Data validation  

The database is now ready for advanced SQL analysis including:
- Window functions (LAG, RANK, NTILE)
- Recursive CTEs
- Statistical aggregations
- Complex joins
- Business intelligence queries

---

**Version:** 2.0  
**Date:** January 2025  
**Author:** Rodion Barskov  
**Database:** SQLite (Rebrickable LEGO Data)  
**Platform:** Google Colab + Rebrickable API